In [1]:
import pandas as pd
file_path = "2A - Base de Dados - Exportações.xlsx"
df = pd.read_excel(file_path, sheet_name="Registros de Vendas", engine="openpyxl")
df.shape, df.columns.tolist()

((2891, 11),
 ['Unnamed: 0',
  'ID_Pedido',
  'Data_Pedido',
  'ID_Representante',
  'Nome_Representante',
  'ID_Produto',
  'Nome_Produto',
  'Categoria',
  'Quantidade_Vendida',
  'Preço_Unitário',
  'Localidade'])

In [2]:
df.head()


,Unnamed: 0,ID_Pedido,Data_Pedido,ID_Representante,Nome_Representante,ID_Produto,Nome_Produto,Categoria,Quantidade_Vendida,Preço_Unitário,Localidade
0,NaN,#P1000001,2024-01-01,1004,Isabela Carolina,SKU9005,Celular Mod 3,Celulares,32,1800,Alemanha/Europa
1,NaN,#P1000002,2024-01-01,1004,Isabela Carolina,SKU9005,Celular Mod 3,Celulares,25,1800,Alemanha/Europa
2,NaN,#P1000003,2024-01-01,1004,Isabela Carolina,SKU9011,Notebook Mod 3,Computadores,40,4300,Estados Unidos/América do Norte
3,NaN,#P1000004,2024-01-01,1004,Isabela Carolina,SKU9004,Celular Mod 2,Celulares,41,1200,Estados Unidos/América do Norte
4,NaN,#P1000005,2024-01-01,1004,Isabela Carolina,SKU9011,Notebook Mod 3,Computadores,38,4300,Estados Unidos/América do Norte


In [3]:
df = df.drop(columns=["Unnamed: 0"], errors="ignore")

# Renomear para nomes sem acentos / sem espaços
df = df.rename(columns={
    "ID_Pedido":"id_pedido",
    "Data_Pedido":"data_pedido",
    "ID_Representante":"id_representante",
    "Nome_Representante":"nome_representante",
    "ID_Produto":"id_produto",
    "Nome_Produto":"nome_produto",
    "Categoria":"categoria",
    "Quantidade_Vendida":"quantidade_vendida",
    "Preço_Unitário":"preco_unitario",
    "Localidade":"localidade"
})
df.columns


Index(['id_pedido', 'data_pedido', 'id_representante', 'nome_representante',
       'id_produto', 'nome_produto', 'categoria', 'quantidade_vendida',
       'preco_unitario', 'localidade'],
      dtype='object')

In [4]:
str_cols = ["nome_representante","nome_produto","categoria","localidade"]
for c in str_cols:
    df[c] = df[c].astype(str).str.strip()
    df[c] = df[c].str.replace(r"\s+", " ", regex=True)  # múltiplos espaços -> 1
    df[c] = df[c].str.title()  # opcional: deixa em Title Case


In [5]:
df["data_pedido"] = pd.to_datetime(df["data_pedido"], errors="coerce", dayfirst=True)
# Ver quantos viraram NaT (datas inválidas)
invalid_dates = df[df["data_pedido"].isna()]
print("Registros com data inválida:", len(invalid_dates))
invalid_dates.head()


Registros com data inválida: 0


,id_pedido,data_pedido,id_representante,nome_representante,id_produto,nome_produto,categoria,quantidade_vendida,preco_unitario,localidade


In [6]:
# remover símbolos de moeda e ajustar vírgula -> ponto se necessário
df["preco_unitario"] = (
    df["preco_unitario"]
    .astype(str)
    .str.replace(r"[^\d\-,\.]", "", regex=True)  # remove símbolos não numéricos
    .str.replace(",", ".", regex=False)          # vírgula decimal -> ponto
)

df["preco_unitario"] = pd.to_numeric(df["preco_unitario"], errors="coerce")
df["quantidade_vendida"] = pd.to_numeric(df["quantidade_vendida"], errors="coerce").fillna(0).astype(int)


In [7]:
df[["preco_unitario","quantidade_vendida"]].isna().sum()


preco_unitario        0
quantidade_vendida    0
dtype: int64

In [8]:
# mostrar amostra de valores estranhos
df.query("quantidade_vendida <= 0 or preco_unitario <= 0").head()

# remover registros sem preço ou sem quantidade (opção)
df = df[ (df["quantidade_vendida"] > 0) & (df["preco_unitario"].notna()) ]


In [9]:
dups = df[df.duplicated(subset=["id_pedido"], keep=False)].sort_values("id_pedido")
print("Duplicados por id_pedido:", dups.shape[0])
# Inspecione e decida: somar quantidades/valores ou manter primeiro


Duplicados por id_pedido: 0


In [10]:
df = df.drop_duplicates()


In [11]:
df["faturamento"] = df["quantidade_vendida"] * df["preco_unitario"]

# criar ano, mes, ano_mes
df["ano"] = df["data_pedido"].dt.year
df["mes"] = df["data_pedido"].dt.month
df["ano_mes"] = df["data_pedido"].dt.to_period("M").astype(str)


In [12]:
# IQR
Q1 = df["faturamento"].quantile(0.25)
Q3 = df["faturamento"].quantile(0.75)
IQR = Q3 - Q1
limite_sup = Q3 + 1.5*IQR
outliers = df[df["faturamento"] > limite_sup]
len(outliers), outliers.head()


(198,
     id_pedido data_pedido  id_representante nome_representante id_produto  \
 12  #P1000013  2024-01-01              1001        Maria Silva    SKU9011   
 26  #P1000027  2024-01-03              1001        Maria Silva    SKU9011   
 29  #P1000030  2024-01-03              1001        Maria Silva    SKU9011   
 33  #P1000034  2024-01-03              1001        Maria Silva    SKU9011   
 36  #P1000037  2024-01-03              1001        Maria Silva    SKU9011   
 
       nome_produto     categoria  quantidade_vendida  preco_unitario  \
 12  Notebook Mod 3  Computadores                  90            4300   
 26  Notebook Mod 3  Computadores                 100            4300   
 29  Notebook Mod 3  Computadores                  98            4300   
 33  Notebook Mod 3  Computadores                 100            4300   
 36  Notebook Mod 3  Computadores                  91            4300   
 
                          localidade  faturamento   ano  mes  ano_mes  
 12         

In [13]:
# Exemplo de padronização de localidade
map_local = {"Brasil":"Brazil", "Br":"Brazil", "United States":"United States"}  # estenda conforme necessário
df["localidade"] = df["localidade"].replace(map_local)


In [14]:
df["localidade"].value_counts().head(30)


localidade
Estados Unidos/América Do Norte    783
China/Ásia                         633
Japão/Ásia                         301
França/Europa                      283
Alemanha/Europa                    214
Rússia/Europa                      189
Itália/Europa                      178
Canadá/América Do Norte            151
Portugal/Europa                     64
Índia/Ásia                          63
Espanha/Europa                      32
Name: count, dtype: int64

In [15]:
# tipos e nulos
print(df.info())

# estatísticas básicas
print(df[["quantidade_vendida","preco_unitario","faturamento"]].describe())

# total faturamento por ano (sanity check)
print(df.groupby("ano")["faturamento"].sum())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2891 entries, 0 to 2890
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   id_pedido           2891 non-null   object        
 1   data_pedido         2891 non-null   datetime64[ns]
 2   id_representante    2891 non-null   int64         
 3   nome_representante  2891 non-null   object        
 4   id_produto          2891 non-null   object        
 5   nome_produto        2891 non-null   object        
 6   categoria           2891 non-null   object        
 7   quantidade_vendida  2891 non-null   int64         
 8   preco_unitario      2891 non-null   int64         
 9   localidade          2891 non-null   object        
 10  faturamento         2891 non-null   int64         
 11  ano                 2891 non-null   int32         
 12  mes                 2891 non-null   int32         
 13  ano_mes             2891 non-null   object      

In [16]:
df.to_csv("exportacoes_preparadas_v1.csv", index=False)
# e salvar versão Excel se preferir
df.to_excel("exportacoes_preparadas_v1.xlsx", index=False)


In [17]:
# por representante
df.groupby("nome_representante").agg(total_faturamento=("faturamento","sum"),
                                     total_pedidos=("id_pedido","nunique"),
                                     media_ticket=("faturamento","mean")).reset_index().to_csv("agg_representante.csv", index=False)

# por produto
df.groupby("nome_produto").agg(total_faturamento=("faturamento","sum"),
                               total_quantidade=("quantidade_vendida","sum")).sort_values("total_faturamento", ascending=False).to_csv("agg_produto.csv", index=False)

# vendas mensais
df.groupby("ano_mes").agg(faturamento=("faturamento","sum")).reset_index().to_csv("vendas_mensais.csv", index=False)
